<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/NER/ner1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets evaluate seqeval

  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=714ca0b7bd5acf9bb18f569ed7f67bab8cfb9d7135da3d8eb68a83f23c79

1. 載入相關套件

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset

2使用數據集

In [4]:
ner_datasets = load_dataset("peoples_daily_ner",cache_dir='./data',trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

peoples_daily_ner.py:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

The repository for peoples_daily_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/peoples_daily_ner.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/20865 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2319 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4637 [00:00<?, ? examples/s]

In [5]:
#查看第一筆資料
ner_datasets['train'][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [6]:
#查看ner_tags,5,6,所代表的意思
ner_datasets['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [7]:
#取得label_list
label_list = ner_datasets['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

3數據預處理

In [8]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [9]:
#不正確,這是以字為單位,而不是以句子為單位
tokenizer(ner_datasets['train'][0]["tokens"])

{'input_ids': [[101, 3862, 102], [101, 7157, 102], [101, 3683, 102], [101, 6612, 102], [101, 1765, 102], [101, 4157, 102], [101, 1762, 102], [101, 1336, 102], [101, 7305, 102], [101, 680, 102], [101, 7032, 102], [101, 7305, 102], [101, 722, 102], [101, 7313, 102], [101, 4638, 102], [101, 3862, 102], [101, 1818, 102], [101, 511, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [10]:
tokenizer(ner_datasets['train'][0]['tokens'], is_split_into_words=True) #必需使用is_split_into_words

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
#英文的拆字,注意數量(10673, 12865, 12921, 8181, 8681)
res = tokenizer('interesting word')
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [12]:
# 0,0,0,0代表第1個字
# 1代表第2個字
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [14]:
def process_function(examples):
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels

    return tokenized_examples


In [15]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [ ]:
from pprint import pprint
pprint(tokenized_datasets['train'][0])

4建立模型

In [17]:
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=len(label_list)) #預設是2,現在是7個

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.num_labels

7

In [19]:
seqeval = evaluate.load('seqeval')
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [20]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    true_predictions =[
        [label_list[p] for p,l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": result["overall_f1"]
    }

配置訓練參數

In [25]:
args = TrainingArguments(
    output_dir = "models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps = 50,
    num_train_epochs=3,
    report_to='none'
)

建立訓練器

In [28]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
)

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.027800,0.019799,0.949581
2,0.012600,0.017106,0.956927
3,0.005900,0.019187,0.959242


TrainOutput(global_step=981, training_loss=0.027933440821494047, metrics={'train_runtime': 1365.0072, 'train_samples_per_second': 45.857, 'train_steps_per_second': 0.719, 'total_flos': 3940951205762142.0, 'train_loss': 0.027933440821494047, 'epoch': 3.0})

In [30]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

{'eval_loss': 0.022736283019185066,
 'eval_f1': 0.9504259743744549,
 'eval_runtime': 39.9411,
 'eval_samples_per_second': 116.096,
 'eval_steps_per_second': 0.926,
 'epoch': 3.0}

9模型預測

In [32]:
from transformers import pipeline

ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
ner_pipe("小明在台北上班")

Device set to use cuda:0


[{'entity_group': 'LABEL_0',
  'score': 0.68817765,
  'word': '小',
  'start': 0,
  'end': 1},
 {'entity_group': 'LABEL_2',
  'score': 0.45806122,
  'word': '明',
  'start': 1,
  'end': 2},
 {'entity_group': 'LABEL_0',
  'score': 0.999788,
  'word': '在',
  'start': 2,
  'end': 3},
 {'entity_group': 'LABEL_5',
  'score': 0.9991885,
  'word': '台',
  'start': 3,
  'end': 4},
 {'entity_group': 'LABEL_6',
  'score': 0.9985606,
  'word': '北',
  'start': 4,
  'end': 5},
 {'entity_group': 'LABEL_0',
  'score': 0.9997852,
  'word': '上 班',
  'start': 5,
  'end': 7}]

In [33]:
model.config

BertConfig {
  "_name_or_path": "google-bert/bert-base-chinese",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_em

In [37]:
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "google-bert/bert-base-chinese",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "ab

In [40]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
ner_pipe("徐國堂在台北上班")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.99963206,
  'word': '徐 國 堂',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.9989937,
  'word': '台 北',
  'start': 4,
  'end': 6}]

In [41]:
tokenizer("徐國堂在台北上班")

{'input_ids': [101, 2528, 1751, 1828, 1762, 1378, 1266, 677, 4408, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [42]:
tokenizer.decode([2528, 1751, 1828])

'徐 國 堂'

In [43]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
res = ner_pipe("徐國堂在台北上班")
res

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.99963206,
  'word': '徐 國 堂',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.9989937,
  'word': '台 北',
  'start': 4,
  'end': 6}]

In [44]:
ner_result = {}
x = '徐國堂在台北上班'
for r in res:
  if r["entity_group"] not in ner_datasets:
    ner_result[r['entity_group']] = []
  ner_result[r['entity_group']].append(x[r['start']:r['end']])
ner_result

{'PER': ['徐國堂'], 'LOC': ['台北']}